In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split as tts
import matplotlib.pyplot as plt

from os import listdir
from PIL import Image
from sklearn.model_selection import train_test_split as tts

# Estudio de la Aruitectura de Méndez con un dataset grande

Busca diferenciar letras de caracteres
Problema actual: Se come toda la RAM 


In [ ]:
data_path = "../input/english-handwritten-characters-dataset"
#../input/english-handwritten-characters-dataset/Img/img001-001.png
dataset = pd.read_csv(data_path + '/english.csv').sample(frac=0.1, random_state=1)
print(len(dataset["label"]))
dataset.head()

In [ ]:
listaImagenes = [np.asarray(Image.open("../input/english-handwritten-characters-dataset/"+nombre)).flatten() for nombre in dataset["image"]]
labels=dataset["label"]

In [ ]:
entradas=np.array(listaImagenes)

In [ ]:
marca=labels.str.isdigit()
salidas=np.array(marca.astype(np.int))
print(salidas)

In [ ]:
xTrain,xTest,yTrain,yTest = tts(entradas,
                                 salidas,
                                 test_size=0.2,
                                 random_state=100)
yTrain=yTrain.reshape(1,yTrain.shape[0])
yTest=yTest.reshape(1,yTest.shape[0])

In [ ]:
def inicializarModelo(x):
    w=np.zeros((x.shape[1],1))
    b=0
    return w,b

def propagacionAdelante(w,b,x):
    z=np.dot(w.T,x.T)+b
    a=1/(1+np.exp(-z))
    return a

def calcularCosto(y,a):
    costo=-np.sum(y*np.log(a)+(1-y)*np.log(1-a))/y.shape[1]
    return costo

def propagacionAtras(w,b,x,y,a,aprendizaje):
    m=y.shape[1]
    dw=np.matmul(x.T,(a-y).T)/m
    db=np.sum(a-y)/m
    w=w-aprendizaje*dw
    b=b-aprendizaje*db
    return w,b

def predecir(w, b, x):
    a=propagacionAdelante(w,b,x)
    yPred=(a>0.5).astype(int)
    return yPred

def calcularExactitud(yPred,y):
    error=np.abs(yPred-y)
    exactitud=100*(1-np.mean(error))
    return round(exactitud,2)

def graficarCostos(costos):
    plt.figure(figsize=(8,6))
    plt.plot(costos['iteracion'],costos['costos'])
    plt.title('Funcion de costo vs iteraciones',size=20)
    plt.xlabel('Iteraciones',size=14,weight=1000)
    plt.ylabel('Costo (error)',size=14,weight=1000)
    plt.xticks(costos['iteracion'],size=14,rotation=-90)
    plt.yticks(size=14)
    plt.grid(color='powderblue',linewidth=2)
    plt.show() 

def guardarModelo(nombreArchivo,w,b):
    modelo=",".join(w.flatten().astype(str))+","+str(b)
    archivo=open(nombreArchivo,"w")
    archivo.write(modelo)
    archivo.close()
    
def cargarModelo(nombreArchivo):
    archivo=open(nombreArchivo,'r')
    modelo= np.array(archivo.readline().split(',')).astype(float)
    archivo.close()
    w=modelo[0:-1].reshape(modelo.shape[0]-1,1)
    b=modelo[-1]
    return w,b
   
def mapSalidas(imagen):
    if imagen.startswith('c'):
        return 0
    else:
        return 1
    
def entrenarModelo(iteraciones,aprendizaje,guardar,x,y):
    w,b=inicializarModelo(x)
    costos={'iteracion':[],'costos':[]} 
    for i in range(iteraciones):
        a=propagacionAdelante(w,b,x)
        w,b=propagacionAtras(w,b,x,y,a,aprendizaje) 
        if i%guardar==0:
            costo=calcularCosto(y,a)
            costos['iteracion'].append(i)
            costos['costos'].append(costo)
            print ("Costo en iteracion {}: {}".format(i,costo))  
    return w,b,costos

In [ ]:
w,b,costos=entrenarModelo(
    iteraciones=1000,
    aprendizaje=0.0000000000005,
    guardar=20,
    x=xTrain,
    y=yTrain)

In [ ]:
yPredTrain=predecir(w,b,xTrain) 
exactitud=calcularExactitud(yPredTrain,yTrain)
print("Exactitud modelo/xTrain: {}%".format(exactitud))
yPredTest=predecir(w,b,xTest)
exactitud=calcularExactitud(yPredTest,yTest)
print("Exactitud modelo/xTest: {}%".format(exactitud))

In [ ]:
guardarModelo("modeloImagenes.txt",w,b)